In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
spark.version

'3.5.0'

In [4]:
!ls -lh /home/jovyan/work/data/fhv_tripdata_2019-10.csv

-rw-r--r-- 1 jovyan users 115M Mar  6 15:54 /home/jovyan/work/data/fhv_tripdata_2019-10.csv


In [5]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])

In [6]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/fhv_tripdata_2019-10.csv')

df = df.repartition(6)

In [7]:
# !sudo chmod -R 775 /home/jovyan/work/data/pq/fhv/2019/10/

[sudo] password for jovyan: 


In [ ]:
df.write.parquet('data/pq/fhv/2019/10/', compression='snappy').mode('overwrite').save()

In [7]:
!ls -lh data/pq/fhv/2019/10/

total 37M
-rw-r--r-- 1 jovyan users 6.1M Mar  6 17:00 part-00000-61956eba-007f-4a6a-98d6-d6fb3040f4f5-c000.snappy.parquet
-rw-r--r-- 1 jovyan users 6.1M Mar  6 17:00 part-00001-61956eba-007f-4a6a-98d6-d6fb3040f4f5-c000.snappy.parquet
-rw-r--r-- 1 jovyan users 6.1M Mar  6 17:00 part-00002-61956eba-007f-4a6a-98d6-d6fb3040f4f5-c000.snappy.parquet
-rw-r--r-- 1 jovyan users 6.1M Mar  6 17:00 part-00003-61956eba-007f-4a6a-98d6-d6fb3040f4f5-c000.snappy.parquet
-rw-r--r-- 1 jovyan users 6.1M Mar  6 17:00 part-00004-61956eba-007f-4a6a-98d6-d6fb3040f4f5-c000.snappy.parquet
-rw-r--r-- 1 jovyan users 6.1M Mar  6 17:00 part-00005-61956eba-007f-4a6a-98d6-d6fb3040f4f5-c000.snappy.parquet
-rw-r--r-- 1 jovyan users    0 Mar  6 17:00 _SUCCESS


In [8]:
df = spark.read.parquet('data/pq/fhv/2019/10/')

**Q3**: How many taxi trips were there on February 15?

In [9]:
from pyspark.sql import functions as F

In [10]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

62610

In [11]:
df.createOrReplaceTempView('fhv_2019_10')

/usr/local/spark/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [12]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhv_2019_10
WHERE
    to_date(pickup_datetime) = '2019-10-15';
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+


**Q4**: Longest trip for each day

In [13]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [32]:
df \
  .withColumn('duration', (df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long'))/3600) \
  .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
  .groupBy('pickup_date') \
  .agg(F.round(F.max('duration'), 1).alias('max(duration)')) \
  .orderBy('max(duration)', ascending=False) \
  .limit(5) \
  .show() 

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2019-10-28|     631152.5|
| 2019-10-11|     631152.5|
| 2019-10-31|      87672.4|
| 2019-10-01|      70128.0|
| 2019-10-17|       8794.0|
+-----------+-------------+


In [27]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    round(MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600), 1) AS duration
FROM 
    fhv_2019_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+--------+
|pickup_date|duration|
+-----------+--------+
| 2019-10-28|631152.5|
| 2019-10-11|631152.5|
| 2019-10-31| 87672.4|
| 2019-10-01| 70128.0|
| 2019-10-17|  8794.0|
| 2019-10-26|  8784.2|
| 2019-10-30|  1464.5|
| 2019-10-25|  1056.8|
| 2019-10-02|   769.2|
| 2019-10-23|   745.6|
+-----------+--------+


**Q5**: Most frequent `dispatching_base_num`

How many stages this spark job has?



In [17]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1)
FROM 
    fhv_2019_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B01145|   47548|
|              B00256|   43234|
|              B00856|   36778|
|              B03016|   34985|
|              B01239|   33586|
+--------------------+--------+


In [18]:
df \
    .groupBy('dispatching_base_num') \
        .count() \
    .orderBy('count', ascending=False) \
    .limit(5) \
    .show()

+--------------------+-----+
|dispatching_base_num|count|
+--------------------+-----+
|              B01145|47548|
|              B00256|43234|
|              B00856|36778|
|              B03016|34985|
|              B01239|33586|
+--------------------+-----+


**Q6**: Least frequent pickup location zone

In [19]:
df_zones = spark.read.parquet('zones')

In [20]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [21]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [22]:
df_zones.registerTempTable('zones')

In [24]:
spark.sql("""
SELECT
    CONCAT(pul.Zone, ' / ', dol.Zone) AS pu_do_pair,
    COUNT(1)
FROM 
    fhv_2019_10 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+--------------------+--------+
|          pu_do_pair|count(1)|
+--------------------+--------+
|             NV / NV|  341803|
|             NV / NA|   38677|
|  NV / East New York|   30540|
|NV / Williamsbrid...|   23154|
|NV / Washington H...|   21667|
+--------------------+--------+


In [35]:
spark.sql("""
SELECT
    zones. Zone as zone,
    COUNT (1)
FROM
    fhv_2019_10, zones
WHERE 
    fhv_2019_10.PUlocationID = zones.LocationID
GROUP BY 1
ORDER BY 2 ASC
LIMIT 5;
""").show()

+--------------------+--------+
|                zone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
+--------------------+--------+
